In [ ]:
import os
import glob
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql import functions
from  pyspark.sql.types import StructType, StructField, StringType, IntegerType,LongType
from datetime import datetime
import time 
from pyspark.sql.functions import col, split, when, size,count,row_number,desc,trim,regexp_replace,lit
import gc
# Define the window specification

os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
print("completed")



file_pattern = r"D:/DataInc-New/Linkedin816MSplittedCSV/linkedin_db-00*.csv"
#file_pattern = r"D:/DataInc/Linkedin816m-Splitted/LinkedInComplete.csv_split_*.csv"
print(file_pattern)
files = glob.glob(file_pattern)

print(f"Number of matched files: {len(files)}")

print(files)



In [ ]:

def read_files(files):
    try:
        # Initialize SparkSession for local mode
        spark = SparkSession.builder.appName("LocalPySparkTest").config("spark.driver.extraClassPath", "C:/Users/****/Downloads/postgresql-42.7.4.jar")\
        .config("spark.sql.shuffle.partitions", "480") \
        .config("spark.driver.memory", "200g") \
        .config("spark.driver.maxResultSize", "10g")\
        .master("local[*]").getOrCreate()  #.config("spark.driver.memory", "63g")\.config("spark.driver.maxResultSize", "60g") \
        
        # Define schema
        shema = StructType([
        StructField("id", LongType(), True),
        StructField("Name", StringType(), True),
        StructField("Email", StringType(), True),
        StructField("PhoneNumber", StringType(), True),
        StructField("LinkedIn", StringType(), True),
        StructField("Title", StringType(), True),
        StructField("Company Name", StringType(), True),
        StructField("CompanyPhone", StringType(), True),
        StructField("Website", StringType(), True),
        StructField("Website2", StringType(), True),
        StructField("Facebook", StringType(), True),
        StructField("Twitter", StringType(), True),
        StructField("LinkedinCompanyPage", StringType(), True),
        StructField("Country", StringType(), True)
        ])


        columns = [
        "first_name", "last_name", "gender", "noOfEmails", "emails", "phone_numbers",
        "job_title", "job_title_role", "job_title_sub_role", "job_title_levels",
        "job_company_name", "job_company_website", "job_company_size", "job_company_founded",
        "job_company_industry", "inferred_salary", "inferred_years_experience", "industry",
        "skills", "interests", "job_company_location_name", "job_company_location_locality",
        "job_company_location_metro", "job_company_location_region", "job_company_location_geo",
        "job_company_location_street_address", "job_company_location_postal_code",
        "job_company_location_country", "job_company_location_continent", "location_name",
        "location_locality", "location_metro", "location_region", "location_country",
        "location_continent", "location_street_address", "location_postal_code", "location_geo",
        "location_names", "street_addresses", "regions", "countries", "experience_companies_names",
        "experience_companies_industries", "experience_companies_job_titles",
        "experience_companies_job_roles", "experience_companies_job_sub_roles",
        "experience_companies_job_levels", "linkedin_url", "facebook_url", "twitter_url",
        "job_company_linkedin_url", "job_company_facebook_url", "job_company_twitter_url",
        "education_degrees", "education_majors", "profiles", "linkedin_connections",
        "certifications", "languages","unknown"
        ]
    
        # Read the CSV file into a DataFrame
        #df = spark.read.csv(file_path, header=True, inferSchema=True)
        print("Spark started reading the file")
        start_time = datetime.now()
        dfs=[spark.read.format("csv").option("header", "false").option("inferSchema", "false").option("delimiter", ",") \
        .option("quote", "\"").option("escape", "\\") \
        .option("lineSep", "\n")\
        #.schema(shema)\
        .load(f) for f in files]
        # Print success message
        df = dfs[0]
        for other_df in dfs[1:]:
            df = df.union(other_df)
        df = df.toDF(*columns)
        
        # Record the end time
        end_time = datetime.now()
        print(f"Process ended at: {end_time}")
        
        # Calculate the duration
        duration = end_time - start_time
        print(f"Total duration: {duration}")
        print("PySpark job ran successfully on the local machine.")
        return df,spark
    
    except Exception as e:
        # Print error message if job fails
        print(f"PySpark job failed with error: {e}")

print('read cell execution success')

In [3]:
def find_dups(df):
    
    windowSpec = Window.partitionBy("linkedin_url").orderBy(col("emails").asc(),col("phone_numbers").asc())
    df1 = df.withColumn("row_number", row_number().over(windowSpec)).filter(col('row_number')==1).drop("row_number")#.filter(col("LinkedInURL").isNotNull())
    print("deleted duplicate and null records")
    return df1

In [ ]:
def trimmed_and_split(df1):
    #windowSpec = Window.partitionBy("LinkedIn").orderBy(col("id").asc())
    #df1 = df.withColumn("row_number", row_number().over(windowSpec)).filter(col('row_number')==1).drop("row_number")#.filter(col("LinkedInURL").isNotNull())
    #print("deleted duplicate and null records")
    #df1.select(['LinkedIn']).show(truncate=False)
    # Trim only string columns
    for column, dtype in df1.dtypes:
        if dtype == "string":
            df1 = df1.withColumn(column, trim(col(column)))
    print("data trimmed successfully")
    
    #df_name_split=df1.withColumn("Fname", trim(split(col("Name"), " ").getItem(0))) \
    #           .withColumn("Lname", trim(split(col("Name"), " ", 2).getItem(1)))
    
    
    #df_final=df_name_split.select('*')
    """(['id',
     'Name','Fname',
     'Lname',
     'Email',
     'PhoneNumber',
     'LinkedIn',
     'Title',
     'Company Name',
     'CompanyPhone',
     'Website',
     'Website2',
     'Facebook',
     'Twitter',
     'LinkedinCompanyPage',
     'Country',
      'batch_count'                            
     ])"""
    return df1

print('trim cell execution success')

In [5]:
def deriv_colum(df_name):
    df_email_split = df_name.withColumn("email_array", split(col("emails"), ","))
    
    # Derive WEmail (index 0) and PEmail (index 1)
    df_emails = df_email_split \
        .withColumn("WEmail", when(size(col("email_array")) > 0, trim(col("email_array")[0])).otherwise(None)) \
        .withColumn("PEmail", when(size(col("email_array")) > 1, trim(col("email_array")[1])).otherwise(None)).drop("email_array")
    #df_emails.select(["email_list","WEmail","PEmail"]).show(200,truncate=False)
    
    
    
    df_split = df_emails.withColumn("phone_array", split(col("phone_numbers"), ","))
    
    # Derive WPhone, HPhone, and MPhone with trimming
    df_phones = df_split \
        .withColumn("WPhone", when(size(col("phone_array")) > 0, trim(col("phone_array")[0])).otherwise(None)) \
        .withColumn("HPhone", when(size(col("phone_array")) > 1, trim(col("phone_array")[1])).otherwise(None)) \
        .withColumn("MPhone", when(size(col("phone_array")) > 2, trim(col("phone_array")[2])).otherwise(None)).drop("phone_array")
    
    
    # Show the result
    #df_phones.select("phone_list", "WPhone", "HPhone", "MPhone").show(200,truncate=False)
    print("emails and phone numbers and names are transformed")
    return df_phones

In [6]:

def write_postgres(df_final,batch):

    print("clean up start")
    for column in df_final.columns:
        df_final = df_final.withColumn(
            column,
            when((col(column).rlike("\\x00") |  (col(column) == "")), None).otherwise(col(column))
        )

    jdbc_url = "jdbc:postgresql://localhost:5432/postgres"  # Replace with your details
    connection_properties = {
        "user": "postgres",         # Replace with your username
        "password": "*******", # Replace with your password
        "driver": "org.postgresql.Driver"
    }
    start_time = datetime.now()
    
    print(f"Process started at: {start_time}")
    
    df_final=df_final.withColumn('batch_count',lit(batch))
    df_final.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "LINKEDIN816_DATA").options(**connection_properties) \
        .mode("append").save()  # Use 'overwrite', 'append', or 'ignore' based on your use case
    
    # Record the end time
    end_time = datetime.now()
    print(f"Process ended at: {end_time}")
    
    # Calculate the duration
    duration = end_time - start_time
    print(f"Total duration: {duration}")
    print("data has been written to master table in postgress")


In [ ]:
# Directory containing the files
folder_path = r"D:/DataInc-New/Linkedin816MSplittedCSV"

# List all files in the directory
all_files = sorted([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".csv")])

# Function to process files in batches
def process_files_in_batches(files, batch_size):
    total_files = len(files)
    for i in range(0, total_files, batch_size):
        batch_files = files[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1}: {len(batch_files)} files")
        batch_count=(i // batch_size + 1)
        #print(batch_files)
        #if batch_count>18:
        df_read,spark=read_files(batch_files)
        #df_read.show()
        print('files read df successfull')
        
        der_df=deriv_colum(df_read)
        #df_trimmed.show()
        #dup_df=find_dups(df_read)
        #dup_df.show()
        df_trimmed=trimmed_and_split(der_df)
        print('trim successfull')
        wr_df=write_postgres(df_trimmed,batch_count)
        print('write df successfull')
        df_read.unpersist(blocking=True) if df_read.is_cached else None
        df_trimmed.unpersist(blocking=True) if df_trimmed.is_cached else None
        der_df.unpersist(blocking=True) if df_read.is_cached else None
        wr_df.unpersist(blocking=True) if df_read.is_cached else None
        print("unpersisted process")
        #return wr_df
        # Trigger garbage collection
        gc.collect()
        print("garbage  process")
        # Clear temporary cache
        spark.catalog.clearCache()
        print("catalog   process")
        spark.stop()
        #else:
        #print(f"batch count is skipped {batch_count}")
        
        


process_files_in_batches(all_files,200)

In [ ]:


all_configs = spark.sparkContext.getConf().getAll()

# Print all configurations
for key, value in all_configs:
    print(f"{key}: {value}")